In [226]:
import pandas as pd

# Assuming 'solar_data' and 'weather_data' are already loaded and have a 'timestamp' column
# Load sunrise and sunset data
sun_data = pd.read_excel('../DATA2/sunrise-sunset.xlsx')

sun_data.head()

,datum,Opkomst,Op ware middag,Ondergang
0,2023-01-01,08:45:00,12:46:00,16:47:00
1,2023-01-02,08:45:00,12:46:00,16:48:00
2,2023-01-03,08:45:00,12:47:00,16:49:00
3,2023-01-04,08:44:00,12:47:00,16:51:00
4,2023-01-05,08:44:00,12:48:00,16:52:00


In [227]:
import pandas as pd
# load in weatherIWillUse.csv
weather = pd.read_csv('../DATA2/weatherIWillUse.csv')
# FID is like this:"2023-02-28 23:00:00+00:00" but it should be like this:"2023-02-28 23:00"
weather['FID'] = weather['FID'].str.slice(0, 16)
# now its liek this :"2023-03-01T09:00"
# save csv
weather.head()

,FID,temp,pressure,cloudiness,humidity_relative
0,2023-03-01 09:00,4.9,1029.7,0.0,54.3
1,2023-03-01 09:00,2.1,1029.3,3.0,73.7
2,2023-03-01 09:00,3.3,1029.6,1.0,84.2
3,2023-03-01 09:00,1.6,1029.4,3.0,70.9
4,2023-03-01 10:00,3.0,1028.8,3.0,60.8


In [228]:
# load in solar_per_uur.csv
solar = pd.read_csv('../DATA2/solar_per_uur.csv')
solar.head()
# the data is like this:"	2023-03-11 15:00:10.160454+00:00" round the seconds to 0
solar['timestamp'] = solar['timestamp'].str.split('.').str[0]
solar['timestamp'] = solar['timestamp'].str[:-3]    
solar.head()

# timestamp,kwh,daily_kwh
aggregated_data = solar.groupby('timestamp').agg({
    'kwh': 'max',
    'daily_kwh': 'sum'
}).reset_index()
# save aggregated_data.csv
aggregated_data.to_csv('../DATA2/aggregated_data.csv', index=False)
aggregated_data.describe()

,kwh,daily_kwh
count,7850.000000,7850.000000
mean,1279.143950,0.255646
std,658.925824,0.454309
min,0.540000,0.000000
25%,720.148450,0.000000
50%,1458.486200,0.003100
75%,1897.632025,0.281498
max,2043.015400,2.098000


In [229]:
# merge weather and solar on timestamp for solar and FID for weather
merged = pd.merge(weather, solar, left_on='FID', right_on='timestamp')
merged.head()
# drop timestamp
merged = merged.drop(columns=['timestamp'])

# rename daily_kwh to hourly_kwh
merged = merged.rename(columns={'daily_kwh': 'hourly_kwh'})
# drop kwh
merged = merged.drop(columns=['kwh'])

# make a new column date now it looks like this:"2023-03-11 15:00" but it should be like this:"2023-03-11"
merged['date'] = merged['FID'].str.split(' ').str[0]
# make a new column time now it looks like this:"2023-03-11 15:00" but it should be like this:"15:00"
merged['time'] = merged['FID'].str.split(' ').str[1]
# save to csv
merged.to_csv('../DATA2/merged.csv', index=False)

merged.describe()
merged.head()

,FID,temp,pressure,cloudiness,humidity_relative,hourly_kwh,date,time
0,2023-03-11 15:00,7.3,1011.9,2.4,48.4,0.54,2023-03-11,15:00
1,2023-03-11 15:00,6.3,1012.0,6.0,84.2,0.54,2023-03-11,15:00
2,2023-03-11 15:00,6.4,1012.0,3.6,64.1,0.54,2023-03-11,15:00
3,2023-03-11 15:00,7.4,1012.2,3.4,52.0,0.54,2023-03-11,15:00
4,2023-03-11 16:00,6.8,1011.9,3.6,54.4,0.48,2023-03-11,16:00


In [230]:
# turn datum Opkomst Op ware middag and Ondergang into a string
sun_data['datum'] = sun_data['datum'].astype(str)
sun_data['Opkomst'] = sun_data['Opkomst'].astype(str)
sun_data['Op ware middag'] = sun_data['Op ware middag'].astype(str)
sun_data['Ondergang'] = sun_data['Ondergang'].astype(str)

# get rid of the seconds
sun_data['Opkomst'] = sun_data['Opkomst'].str[:-3]
sun_data['Op ware middag'] = sun_data['Op ware middag'].str[:-3]
sun_data['Ondergang'] = sun_data['Ondergang'].str[:-3]



sun_data.head()

,datum,Opkomst,Op ware middag,Ondergang
0,2023-01-01,08:45,12:46,16:47
1,2023-01-02,08:45,12:46,16:48
2,2023-01-03,08:45,12:47,16:49
3,2023-01-04,08:44,12:47,16:51
4,2023-01-05,08:44,12:48,16:52


In [231]:
# merge sun_data and merged on date and datum
merged = pd.merge(merged, sun_data, left_on='date', right_on='datum')
# drop datum
merged = merged.drop(columns=['datum'])

# convert time to datetime
merged['time'] = pd.to_datetime(merged['time'])
# convert Opkomst to datetime
merged['Opkomst'] = pd.to_datetime(merged['Opkomst'])
# convert Op ware middag to datetime
merged['Op ware middag'] = pd.to_datetime(merged['Op ware middag'])
# convert Ondergang to datetime
merged['Ondergang'] = pd.to_datetime(merged['Ondergang'])


# make a new column afterSunrise if time is after Opkomst then True else False
merged['afterSunrise'] = merged['time'] > merged['Opkomst']
# make a new column beforeSunset if time is before Ondergang then True else False
merged['beforeSunset'] = merged['time'] < merged['Ondergang']
# make a new column containswaremiddag if the 2 first numbers of time are the same as the 2 first numbers of Op ware middag then True else False
merged['distanceToWareMiddag'] = abs((merged['time'] - merged['Op ware middag']).dt.total_seconds() / 3600)

# make a new column has sunLight if afterSunrise and beforeSunset are true then True else False
merged['hasSunLight'] = merged['afterSunrise'] & merged['beforeSunset']

# drop date and time
merged = merged.drop(columns=['date', 'time'])

# drop Opkomst, Op ware middag and Ondergang
merged = merged.drop(columns=['Opkomst', 'Op ware middag', 'Ondergang'])

# drop afterSunrise and beforeSunset
merged = merged.drop(columns=['afterSunrise', 'beforeSunset'])

# save to csv
merged.to_csv('../DATA2/merged.csv', index=False)




merged.head()


C:\Users\tomde\AppData\Local\Temp\ipykernel_21804\773006717.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged['time'] = pd.to_datetime(merged['time'])
C:\Users\tomde\AppData\Local\Temp\ipykernel_21804\773006717.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged['Opkomst'] = pd.to_datetime(merged['Opkomst'])
C:\Users\tomde\AppData\Local\Temp\ipykernel_21804\773006717.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged['Op ware middag'] = pd.to_datetime(merged['Op ware middag'])
C:\Users\tomde\AppData\Local\Temp\ipykernel_21804\773006717.py:13: UserWa

,FID,temp,pressure,cloudiness,humidity_relative,hourly_kwh,distanceToWareMiddag,hasSunLight
0,2023-03-11 15:00,7.3,1011.9,2.4,48.4,0.54,2.116667,True
1,2023-03-11 15:00,6.3,1012.0,6.0,84.2,0.54,2.116667,True
2,2023-03-11 15:00,6.4,1012.0,3.6,64.1,0.54,2.116667,True
3,2023-03-11 15:00,7.4,1012.2,3.4,52.0,0.54,2.116667,True
4,2023-03-11 16:00,6.8,1011.9,3.6,54.4,0.48,3.116667,True


In [232]:
forecast = pd.read_csv('../DATA2/forecast.csv')
forecast['timestamp'] = forecast['timestamp'].str.slice(0, 16)
forecast['timestamp'] = pd.to_datetime(forecast['timestamp'])
forecast.head()

,timestamp,temp,pressure,cloudiness,humidity_relative,hasSunLight,distanceToWareMiddag
0,2024-04-19 11:00:00,9.76,997,8.0,87,True,2.7
1,2024-04-19 12:00:00,9.67,993,8.0,87,True,1.7
2,2024-04-19 13:00:00,9.74,997,8.0,86,True,0.7
3,2024-04-19 14:00:00,9.63,1001,8.0,85,True,0.3
4,2024-04-19 15:00:00,9.37,1005,8.0,83,True,1.3


In [233]:
# convert timestamp to string
forecast['timestamp'] = forecast['timestamp'].astype(str)



# make a new column date now it looks like this:"2023-03-11 15:00" but it should be like this:"2023-03-11"
forecast['date'] = forecast['timestamp'].str.split(' ').str[0]
# make a new column time now it looks like this:"2023-03-11 15:00" but it should be like this:"15:00"
forecast['time'] = forecast['timestamp'].str.split(' ').str[1]

# merge sun_data and merged on date and datum
forecast = pd.merge(forecast, sun_data, left_on='date', right_on='datum')
# drop datum
forecast = forecast.drop(columns=['datum'])

# convert time to datetime
forecast['time'] = pd.to_datetime(forecast['time'])
# convert Opkomst to datetime
forecast['Opkomst'] = pd.to_datetime(forecast['Opkomst'])
# convert Op ware middag to datetime
forecast['Op ware middag'] = pd.to_datetime(forecast['Op ware middag'])
# convert Ondergang to datetime
forecast['Ondergang'] = pd.to_datetime(forecast['Ondergang'])


# make a new column afterSunrise if time is after Opkomst then True else False
forecast['afterSunrise'] = forecast['time'] > forecast['Opkomst']
# make a new column beforeSunset if time is before Ondergang then True else False
forecast['beforeSunset'] = forecast['time'] < forecast['Ondergang']
# make a new column containswaremiddag if the 2 first numbers of time are the same as the 2 first numbers of Op ware middag then True else False
forecast['distanceToWareMiddag'] = abs((forecast['time'] - forecast['Op ware middag']).dt.total_seconds() / 3600)

# make a new column has sunLight if afterSunrise and beforeSunset are true then True else False
forecast['hasSunLight'] = forecast['afterSunrise'] & forecast['beforeSunset']

# drop date and time
forecast = forecast.drop(columns=['date', 'time'])

# drop Opkomst, Op ware middag and Ondergang
forecast = forecast.drop(columns=['Opkomst', 'Op ware middag', 'Ondergang'])

# drop afterSunrise and beforeSunset
forecast = forecast.drop(columns=['afterSunrise', 'beforeSunset'])

# save to csv
forecast.to_csv('../DATA2/forecast.csv', index=False)





forecast.head()


C:\Users\tomde\AppData\Local\Temp\ipykernel_21804\3078904618.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  forecast['time'] = pd.to_datetime(forecast['time'])
C:\Users\tomde\AppData\Local\Temp\ipykernel_21804\3078904618.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  forecast['Opkomst'] = pd.to_datetime(forecast['Opkomst'])
C:\Users\tomde\AppData\Local\Temp\ipykernel_21804\3078904618.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  forecast['Op ware middag'] = pd.to_datetime(forecast['Op ware middag'])
C:\Users\tomde\AppData\Local\Temp\ipykernel_21804\307890

,timestamp,temp,pressure,cloudiness,humidity_relative,hasSunLight,distanceToWareMiddag
0,2024-04-19 11:00:00,9.76,997,8.0,87,True,2.7
1,2024-04-19 12:00:00,9.67,993,8.0,87,True,1.7
2,2024-04-19 13:00:00,9.74,997,8.0,86,True,0.7
3,2024-04-19 14:00:00,9.63,1001,8.0,85,True,0.3
4,2024-04-19 15:00:00,9.37,1005,8.0,83,True,1.3
